# Introduction
Currently, the standard procedure within PorePy is to export data to vtu and pvd format for visualization with ParaView. This tutorial explains how to use the 'Exporter'. In particular, it will showcase different ways to address data, how constant-in-time data is handled, and how pvd-files are managed. 

First, an example data set is defined, then the actual exporter is defined, before currently all possible ways to export data are demonstrated.

NOTE: Related but not necessary for this tutorial: it is highly recommended to read the ParaView documentation. 

## Example contact mechanics model for a mixed-dimensional geometry
In order to illustrate the capability and explain the use of the Exporter, we consider a ContactMechanicsBiot model for a two-dimensional fractured geometry. The mixed-dimensional geometry consists of a 2D square and two crossing 1D fractures.

In [2]:
import numpy as np
import porepy as pp

class BiotFractured(pp.ContactMechanicsBiot):

    def create_grid(self) -> None:
        # Define domain
        self.box: Dict = pp.geometry.bounding_box.from_points(
            np.array([[0, 1], [0, 1]])
        )

        # Define fracture network
        # ... pts
        frac_a = np.array([[1 / 2.0, 0.0], [1 / 2.0, 0.5]]).T
        frac_b = np.array([[0.16, 0.2], [1, 0.5]]).T

        intersecting_fractures = True

        if intersecting_fractures:
            frac_pts = np.hstack((frac_a, frac_b))
            frac_connectivity = np.array([[0, 1], [2,3]]).T
        else:
            frac_pts = frac_a
            frac_connectivity = np.array([[0], [1]])

        # ... network
        network_2d = pp.FractureNetwork2d(frac_pts, frac_connectivity, self.box)

        # Generate mixed-dimensional mesh
        mesh_size_frac = self.params.get("mesh_size_frac", 1e-1)
        mesh_size_bound = self.params.get("mesh_size_bound", 1e-1)
        mesh_args = {
            "mesh_size_frac": mesh_size_frac,
            "mesh_size_bound": mesh_size_bound
        }
        self.gb = network_2d.mesh(mesh_args)

        pp.contact_conditions.set_projections(self.gb)

params = {"use_ad": True}
#model = pp.ContactMechanicsBiot(params)
model = BiotFractured(params)
model.prepare_simulation()



Bad key text.latex.preview in file /home/jakub/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/jakub/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /home/jakub/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # when a jpeg is saved, the default quality p

The default data of the model is stored as pp.STATE in the grid bucket. Let's have a look:

In [3]:
# Determine all keys of all states on all subdomains
subdomain_states = []
for g, d in model.gb.nodes():
    subdomain_states += d[pp.STATE].keys()
# The key pp.ITERATE is no actual state.
subdomain_states = set(subdomain_states) - set([pp.ITERATE])
print("Keys of the states defined on subdomains:", subdomain_states)

# Determine all keys of all states on all interfaces
interface_states = []
for g, d in model.gb.edges():
    interface_states += d[pp.STATE].keys()
# The key pp.ITERATE is no actual state.
interface_states = set(interface_states) - set([pp.ITERATE])
print("Keys of the states defined on interfaces:", set(interface_states))   

Keys of the states defined on subdomains: {'p', 'u', 'contact_traction'}
Keys of the states defined on interfaces: {'mortar_u', 'mortar_p'}


## Defining the exporter
The definition of an object of type pp.Exporter, merely a grid bucket as well as a target is required for the output.

In [4]:
exporter = pp.Exporter(model.gb, file_name="file", folder_name="folder")

In the following, we showcase how to use the main subroutines for exporting data:
- write_vtu()
- write_pvd()
The first, addresses the export of data for a specific time step, while the latter, gather the previous exports and collects them in a single file. This allows an easier analysis in ParaView.

## Example 1: Exporting states
Data stored in the grid bucket under 'pp.STATE' can be simply exported by addressing their keys using the routine 'write_vtu()'. We define a dedicated exporter for this task.

In [5]:
exporter_1 = pp.Exporter(model.gb, file_name="example-1", folder_name="exporter-tutorial")
exporter_1.write_vtu(["p", "u", "mortar_p"])

Note, that here all available representation (i.e., on all dimensions) of the states will be exported.

## Example 2: Exporting states on specified grids
Similar to Example 1, we will again export states by addressing their keys, but target only a subset of grids. For instance, we fetch the grids for the subdomains and interface.

NOTE: For now, one has to make sure that subsets of the mixed-dimensional grid contain all grids of a particular dimension.

In [6]:
grids_1d = model.gb.get_grids(lambda g: g.dim == 1).tolist()
grids_2d = model.gb.get_grids(lambda g: g.dim == 2).tolist()
edges_1d = [e for e, d in model.gb.edges() if d["mortar_grid"].dim == 1]

And as a simple example extract the 2D subdomain:

In [7]:
g_2d = grids_2d[0]

We export pressure on all 1D subdomains, displacements on a single 2D subdomain, and the mortars on all interfaces. For this, we use tuples of grid(s) and keys. In order to not overwrite the previous data, we define a new exporter.

In [8]:
exporter_2 = pp.Exporter(model.gb, "example-2", "exporter-tutorial")
exporter_2.write_vtu([(grids_1d, "p"), (g_2d, "u"), (edges_1d, "mortar_p")])

## Example 3: Exporting explicitly defined data
There exists the possibility to export data which is not stored in the grid bucket under 'pp.STATE'. This capability requires defining tuples of (1) a single grid, (2) a key, and (3) the data vector. For example, let's export the cell centers of the 2D subdomain 'g_2d', as well as all interfaces. Again, we define a dedicated exporter for this task.


In [9]:
subdomain_data = [(g_2d, "cc", g_2d.cell_centers)]
interface_data = [
    (edges_1d[i], "cc_e", model.gb.edge_props(edges_1d[i])["mortar_grid"].cell_centers)
    for i in range(len(edges_1d))
]

exporter_3 = pp.Exporter(model.gb, "example-3", "exporter-tutorial")
exporter_3.write_vtu(subdomain_data + interface_data)

## Example 4: Flexibility in the input arguments
The export allows for an arbitrary combination of all previous ways to export data.

In [10]:
exporter_4 = pp.Exporter(model.gb, "example-4", "exporter-tutorial")
exporter_4.write_vtu([(g_2d, "cc", g_2d.cell_centers), (grids_1d, "p"), "u"] + interface_data)

## Example 5: Exporting data in a time series
Data can also be exported in a time series, and the Exporter takes care of managing the file names. The user will only have to prescribe the time step number. Consider a time series consisting of 5 steps. For simplicity, we consider an analogous situation as in Example 1.

In [11]:
exporter_5 = pp.Exporter(model.gb, "example-5", "exporter-tutorial")
for step in range(5):
    # Data may change
    exporter_5.write_vtu(["p", "u", "mortar_p"], time_step=step)

## Example 6: Exporting constant data
The export of both grid and geometry related data as well as heterogeneous material parameters may be of interest. However, these often do only change very seldomly in time or are even constant in time. In order to save storage space, constant data is stored separately. A multirate approach is used to address slowly changing "constant" data, which results in an extra set of output files. Everytime constant data has to be updated (in a time series), the output files are updated as well. 

In [12]:
exporter_6_a = pp.Exporter(model.gb, "example-6-a", "exporter-tutorial")
# Define some 'constant' data
exporter_6_a.add_constant_data([(g_2d, "cc", g_2d.cell_centers)])
for step in range(5):
    # Update the previously defined 'constant' data
    if step == 2:
        exporter_6_a.add_constant_data([(g_2d, "cc", -g_2d.cell_centers)])
    # All constant data will be exported also if not specified
    exporter_6_a.write_vtu(["p", "u", "mortar_p"], time_step=step)

The default is that constant data is always printed to extra files. Since the vtu format requires geometrical and topoligical information on the mesh (points, connectivity etc.), this type of constant data is exported to each vtu file. Depending on the situation, this overhead can be in fact significant. Thus, one can also choose to print the constant data to the same files as the standard data, by setting a keyword when defining the exporter. With a similar setup as in part A, the same output is generated, but managed differently among files.

In [13]:
exporter_6_b = pp.Exporter(model.gb, "example-6-b", "exporter-tutorial", export_constants_separately = False)
exporter_6_b.add_constant_data([(g_2d, "cc", g_2d.cell_centers)])
for step in range(5):
    # Update the previously defined 'constant' data
    if step == 2:
        exporter_6_b.add_constant_data([(g_2d, "cc", -g_2d.cell_centers)])
    # All constant data will be exported also if not specified
    exporter_6_b.write_vtu(["p", "u", "mortar_p"], time_step=step)

## Example 5 revisisted: PVD format
The pvd format collects previously exported data. At every application of 'write_vtu' a corresponding pvd file is generated, gather all 'vtu' files correpsonding to this time step. It is recommended to use the 'pvd' file for analyzing the data in ParaView.

In addition, when considering a time series, it is possible to gather data connected to multiple time steps, and assign the actual time to each time step. Assume, Example 5 corresponds to an adaptive time stepping. We define the actual times, and write collect the exported data from Example 5 in a single pvd file.

In [16]:
times_5 = [0., 0.1, 1., 2., 10.]
exporter_5.write_pvd(times_5)

## Example 7: Exporting data on a single grid
It is also possible to export data without prescribing a grid bucket, but a single grid. In this case, one has to assign the data when writing to vtu. For this, a key and a data array (with suitable size) has to be provided.

In [17]:
exporter_7 = pp.Exporter(g_2d, "example-7", "exporter-tutorial")
exporter_7.write_vtu([("cc", g_2d.cell_centers)])